In [2]:
from requests import post,get
import base64
import os
import json

In [3]:
CLIENT_ID = '92c7e8974c32468bb7f8fdba1d898847'
SECRET_ID = 'a23b031188f34caca5c97bc7851c2192'


In [4]:
def get_token_access():
    auth_string = f"{CLIENT_ID}:{SECRET_ID}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {auth_base64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [5]:
#API SEARCH
import requests
import json

def search_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        #utilizziamo json in modo che i dati possono essere facilmente manipolati e utilizzati 
        data = json.loads(response.content)["artists"]["items"]
        return data[0]
    else:
        print(f"Errore nella richiesta HTTP: {response.status_code}")
        return None  

In [6]:
access_token = get_token_access()
result = search_artist(access_token, "Calcutta")
print(result)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/582KhTHEVOONNQLmQ5612r'}, 'followers': {'href': None, 'total': 660628}, 'genres': ['italian adult pop', 'italian pop', 'rome indie'], 'href': 'https://api.spotify.com/v1/artists/582KhTHEVOONNQLmQ5612r', 'id': '582KhTHEVOONNQLmQ5612r', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebe6f42fb00b849720936df798', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174e6f42fb00b849720936df798', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178e6f42fb00b849720936df798', 'width': 160}], 'name': 'Calcutta ', 'popularity': 67, 'type': 'artist', 'uri': 'spotify:artist:582KhTHEVOONNQLmQ5612r'}


In [7]:
#estraggo le informazioni inerenti al nome, id , url, genere ,follower e immagine della nostra ricerca

artist_name = result["name"]
artist_id = result["id"] #spotify id for the artist
artist_url = result["external_urls"]["spotify"] #the spotify url for the object 
artist_genres = result["genres"]
artist_followers = result["followers"]["total"]
artist_image = result["images"][0]["url"]

In [8]:
import pandas as pd
from IPython.display import Image, display, HTML

#creo un dizionario
artist_data = {
    "NAME": [artist_name],
    "ID": [artist_id],
    "SPOTIFY URL": [artist_url],
    "GENERE": [artist_genres],
    "FOLLOWER": [artist_followers],
    "artist_image_url": [artist_image]
}
#permette di visualizzare il contenuto delle colonne senza troncarle in caso di tot lunghezza
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(artist_data)

image_width = 250
# Visualizza l'immagine cnetrata
image_html = f'<div style="display: flex; justify-content: center;">' \
             f'<img src="{df["artist_image_url"][0]}" alt="Artista" width="{image_width}" style="display: block;"/>' \
             f'</div>'
display(HTML(image_html))

#df drop rimuove la collone artist_image_url e visualizza il dataframe
display(df.drop("artist_image_url", axis=1))

,NAME,ID,SPOTIFY URL,GENERE,FOLLOWER
0,Calcutta,582KhTHEVOONNQLmQ5612r,https://open.spotify.com/artist/582KhTHEVOONNQLmQ5612r,"[italian adult pop, italian pop, rome indie]",660628


In [9]:
#TRACK PIU FAMOSE DI QUEST'ARTISTA

def top_tracks_artist(token, artist_id):
    url=f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=IT"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #utilizziamo json in modo che i dati possono essere facilmente manipolati e utilizzati 
        data = json.loads(response.content)["tracks"]
        return data
    else:
        print(f"Errore nella richiesta HTTP: {response.status_code}")
        return None  


In [10]:
res = top_tracks_artist(access_token, artist_id)
res

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/582KhTHEVOONNQLmQ5612r'},
     'href': 'https://api.spotify.com/v1/artists/582KhTHEVOONNQLmQ5612r',
     'id': '582KhTHEVOONNQLmQ5612r',
     'name': 'Calcutta ',
     'type': 'artist',
     'uri': 'spotify:artist:582KhTHEVOONNQLmQ5612r'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/4fPV5OWBxzBVaYRbXduGtj'},
   'href': 'https://api.spotify.com/v1/albums/4fPV5OWBxzBVaYRbXduGtj',
   'id': '4fPV5OWBxzBVaYRbXduGtj',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b2738acaa7da2fb43a4038d962c6',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e028acaa7da2fb43a4038d962c6',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d000048518acaa7da2fb43a4038d962c6',
     'width': 64}],
   'is_playable': True,
   'name': 'RELAX',
   'release_date': '2023-10-20',
   '

In [11]:
#alcuni dati inerenti ad una singola canzone
artist_uri= res[0]["uri"]
preview_track = res[0]["preview_url"]
number_track = res[0]["track_number"]
popularity_track = res[0]["popularity"]
name_track = res[0]["name"]
link = res[0]["external_urls"]["spotify"]
id_track = res[0]["id"]
duration_ms = res[0]['duration_ms']

print(res[0]["album"]["name"])
print(res[0]["album"]["images"][0]["url"])
print(res[0]["album"]["total_tracks"])

RELAX
https://i.scdn.co/image/ab67616d0000b2738acaa7da2fb43a4038d962c6
11


In [12]:
#andiamo a visualizzare corettamente la playlist

df = pd.DataFrame([{
    'Titolo': song['name'],
    #ci sono 60000 ms in un minuto con la divisione andiamo a prendere i minuti, dopodiche prendiamo il resto dei ms
    #dividendo per 1000 otteniamo i secondi e prendiamo solo le prime due cifre intere
    'Durata': f"{song['duration_ms'] // 60000}:{(song['duration_ms'] % 60000) // 1000:02d}",
    'Nome Album' : song["album"]["name"],
    'Song URL': song['external_urls']['spotify']
} for song in res])


#miglioriamo lo stile di visualizzazione del dataframe
styled_df = df.style.set_properties(**{'text-align': 'left'})\
            .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])\
            .format({'Song URL': '<a href="{}">Listen here</a>'}) #formattazione url come pulsante da cliccare

display(styled_df)


,Titolo,Durata,Nome Album,Song URL
0,2minuti,3:34,RELAX,Listen here
1,Tutti,4:10,RELAX,Listen here
2,Giro con te,3:24,RELAX,Listen here
3,Controtempo,3:44,RELAX,Listen here
4,Cosa mi manchi a fare,2:49,Mainstream (Deluxe Edition),Listen here
5,Oroscopo (feat. Takagi & Ketra) - Ghost track,3:18,Mainstream +,Listen here
6,Loneliness,2:39,RELAX,Listen here
7,SSD,4:23,RELAX,Listen here
8,Preoccuparmi,3:38,RELAX,Listen here
9,Ghiaccioli,3:07,RELAX,Listen here


In [13]:
#PRENDE TUTTI GLI ALBUM FATTI DA UN ARTISTA
def get_artist_album(token, id):
    url=f"https://api.spotify.com/v1/artists/{id}/albums"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "market" : "IT",
        "limit": 10
    }
    response = requests.get(url, headers=headers, params= params)
    if response.status_code == 200:
        #utilizziamo json in modo che i dati possono essere facilmente manipolati e utilizzati 
        data = json.loads(response.content)["items"]
        return data
    else:
        print(f"Errore nella richiesta HTTP: {response.status_code}")
        return None  


In [14]:
res = get_artist_album(access_token, artist_id)
res

[{'album_group': 'album',
  'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/582KhTHEVOONNQLmQ5612r'},
    'href': 'https://api.spotify.com/v1/artists/582KhTHEVOONNQLmQ5612r',
    'id': '582KhTHEVOONNQLmQ5612r',
    'name': 'Calcutta ',
    'type': 'artist',
    'uri': 'spotify:artist:582KhTHEVOONNQLmQ5612r'}],
  'external_urls': {'spotify': 'https://open.spotify.com/album/4fPV5OWBxzBVaYRbXduGtj'},
  'href': 'https://api.spotify.com/v1/albums/4fPV5OWBxzBVaYRbXduGtj',
  'id': '4fPV5OWBxzBVaYRbXduGtj',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2738acaa7da2fb43a4038d962c6',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e028acaa7da2fb43a4038d962c6',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048518acaa7da2fb43a4038d962c6',
    'width': 64}],
  'is_playable': True,
  'name': 'RELAX',
  'release_date': '2023-10-20',
  'releas

In [16]:
df = pd.DataFrame([{
    'image' : album["images"][0]["url"],
    'Nome Album': album['name'],
    'Data di uscita': album['release_date'],
    'Numero di canzoni' : album["total_tracks"],
    'Album url': album['external_urls']['spotify']
} for album in res])

df['Immagine'] = df['image'].apply(lambda x: f'<img src="{x}" alt="Album Cover" style="max-height: 50px; max-width: 50px;">')

# Seleziona solo le colonne desiderate
selected_columns = ['Immagine','Nome Album', 'Data di uscita', 'Numero di canzoni', 'Album url']
df_selected = df[selected_columns]
df_selected = df_selected[df_selected['Numero di canzoni'] != 1] #eliminiamo playlist che hanno solo una canzone
 
# Visualizza il DataFrame con le immagini
display(HTML(df_selected.to_html(escape=False)))

NameError: name 'album_data_frame' is not defined

In [ ]:
def get_artist_related(token, id):
    url=f"https://api.spotify.com/v1/artists/{id}/related-artists"
    headers = {
        "Authorization": f"Bearer {token}"
    }
 
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #utilizziamo json in modo che i dati possono essere facilmente manipolati e utilizzati 
        data = json.loads(response.content)["artists"]
        return data
    else:
        print(f"Errore nella richiesta HTTP: {response.status_code}")
        return None  

In [ ]:
results = get_artist_related(access_token, artist_id)


df = pd.DataFrame([{
    'name' : result["name"],
    'followers' : result["followers"]["total"],
     'image' : f'<img src="{result["images"][0]["url"]}" width="40" height="40">'
} for result in results])



df = df.sort_values(by='followers',ascending=False)
#reset degli indici
df= df.reset_index(drop=True)

display(HTML(df.to_html(escape=False)))


In [ ]:
import matplotlib.pyplot as plt

# Creare una lista dei nomi e una lista dei follower dai dati
names = df['name'].tolist()
followers = df['followers'].tolist()

plt.figure(figsize=(10, 6))
plt.plot(names, followers, marker='o', color='blue', linestyle='-')
plt.title('Popolarità in base ai follower')
plt.xlabel('Artista')
plt.ylabel('Follower')
plt.xticks(rotation=90)
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def get_genres(token):
    url ="https://api.spotify.com/v1/recommendations/available-genre-seeds"
    headers = {
        "Authorization": f"Bearer {token}"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #utilizziamo json in modo che i dati possono essere facilmente manipolati e utilizzati 
        data = json.loads(response.content)["genres"]
        return data
    else:
        print(f"Errore nella richiesta HTTP: {response.status_code}")
        return None  

In [ ]:
res=get_genres(access_token)
res

In [ ]:
import time


def new_dataframe(genres_list, token, limit=50):
    data = {
        'id_track': [],
        'name': [],
        'genre': []
    }

    headers = {
        'Authorization': f'Bearer {token}'
    }

    for genre in genres_list:
        params = {
            'q': f'genre:"{genre}"',
            'type': 'track',
            'limit': limit,
            'market' : 'IT'
        }

        # Esegui la richiesta HTTP GET
        response = requests.get("https://api.spotify.com/v1/search", headers=headers, params=params)

        if response.status_code == 200:
            results = response.json()

            for track in results['tracks']['items']:
                data['id_track'].append(track['id'])
                data['name'].append(track['name'])
                data['genre'].append(genre)

        # Aspetta per 2 secondi prima di effettuare la prossima richiesta
        time.sleep(1)

    return data


genres = [
    'acoustic', 'afrobeat','alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova',
    'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy',
    'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass',
    'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german',
    'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
    'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial',
    'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop',
    'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode',
    'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-pop', 'progressive-house',
    'psych-rock', 'punk', 'punk-rock', 'r-n-b', 'rainy-day', 'reggae', 'reggaeton', 'road-trip', 'rock', 'rock-n-roll',
    'rockabilly', 'romance', 'sad', 'salsa', 'samba', 'sertanejo', 'show-tunes', 'singer-songwriter', 'ska', 'sleep',
    'songwriter', 'soul', 'soundtracks', 'spanish', 'study', 'summer', 'swedish', 'synth-pop', 'tango', 'techno',
    'trance', 'trip-hop', 'turkish', 'work-out', 'world-music']

In [ ]:
res = new_dataframe(genres, access_token, limit=50)

In [ ]:
import csv

def save_data_to_csv(data, file_name):
    with open(file_name, 'w', newline='', encoding='utf-8') as csv_file: 
        fieldnames = ['id_track', 'name', 'genre']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(len(data['id_track'])):
            writer.writerow({
                'id_track': data['id_track'][i],
                'name': data['name'][i],
                'genre': data['genre'][i]
            })


save_data_to_csv(res, 'output.csv')



In [2]:
import requests
import datetime
from flask import Flask, redirect, session, request , jsonify , render_template
import urllib.parse
from datetime import datetime , timedelta
import secrets
from base64 import b64encode


state = secrets.token_urlsafe(16)  # Genera un valore casuale per state

REDIRECT_URI = "http://localhost:5000/callback"
TOKEN_URL = "https://accounts.spotify.com/api/token"
AUTH_URL = "https://accounts.spotify.com/authorize"
API_BASE_URL = "https://api.spotify.com/v1/"

app = Flask(__name__)
app.secret_key = '1299213929x9sdasd9asdsa-2919293'  # You should set your own secret key

@app.route('/')
#decoratore definisce una route per url di base -> applicazione aperta viene eseguita la funzione index
def index():
    return "Welcome to my spotify app <a href='/login'> Login with spotify </a>"
#la funzione index ha una stringa di benvenuto con un link login with spotify reindirizza alla funzione login

@app.route('/login')
def login():
    scope= 'user-read-private user-read-email'
    #scope -> permessi che l'utente deve accettare per andare avanti
    params = { 
            'client_id' : CLIENT_ID,
            'response_type' : 'code',
            'scope' : scope,
            'redirect_uri' : REDIRECT_URI, #dove viene indirizzato dopo aver avuto accesso/negato l'autorizzazione
            'show_dialog' : True,
            'state' : state
    }

    auth_url = f"{AUTH_URL}?{urllib.parse.urlencode(params)}"
    print(auth_url)
    return redirect(auth_url)

#questa funzione gestisce la risposta di spotify dopo che l'utente ha accettato l'autorizzazione
@app.route('/callback')
def callback():
    print(request.args)
    if 'error' in request.args:
        return  request.args["error"]

    if 'code' in request.args:
        req_body = {
            'code': request.args["code"],
            'grant_type': 'authorization_code',
            'redirect_uri' : REDIRECT_URI, #dove viene indirizzato dopo aver avuto accesso/negato l'autorizzazione
            'client_id' : CLIENT_ID,
            'client_secret': SECRET_ID
        }
    
        response = requests.post(TOKEN_URL, data=req_body)
        token_info = response.json()
        print(token_info)
        session['access_token'] = token_info.get('access_token')
        session['refresh_token'] = token_info.get('refresh_token')
        session['expires_at'] = datetime.now().timestamp() + token_info.get('expires_in', 3600)  # Set an appropriate expiration time
        page_content = f"<h1>Token Information:</h1><pre>{token_info.get('access_token')}</pre>"
        page_content += "<form action='/playlists' method='get'>"
        page_content += "<button type='submit'>Playlist</button>"
        page_content += "</form>"

        return page_content

    return "Invalid callback request"


@app.route('/playlists')
def get_playlists():
    if 'access_token' not in session:
        return redirect('/login')

    if datetime.now().timestamp() > session['expires_at']:
        print("TOKEN EXPIRED")
        return redirect('/refresh-token')

    headers = {
        'Authorization' : f"Bearer {session['access_token']}"
    }

    response = requests.get(API_BASE_URL + 'me/playlists', headers=headers)
    playlists = response.json()

    return jsonify(playlists)


@app.route('/refresh-token')
def refresh_token():
    if 'refresh_token' not in session:
        return redirect('/login')
    
    if datetime.now().timestamp() > session['expires_at']:
        req_body = {
            'grant_type': 'refresh_token',  # Correzione: 'grant_type' invece di 'grant-type'
            'refresh_token': session['refresh_token'],
            'client_id': CLIENT_ID,
            'client_secret': SECRET_ID
        }

        response = requests.post(TOKEN_URL, data=req_body)
        new_token_info = response.json()

        session['access_token'] = new_token_info['access_token']
        session['expires_at'] = datetime.now().timestamp() + 10

        return redirect('/playlists')

In [ ]:
if __name__ == '__main__'  :
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Nov/2023 12:00:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 12:00:14] "GET /login HTTP/1.1" 302 -


https://accounts.spotify.com/authorize?client_id=92c7e8974c32468bb7f8fdba1d898847&response_type=code&scope=user-read-private+user-read-email&redirect_uri=http%3A%2F%2Flocalhost%3A5000%2Fcallback&show_dialog=True&state=3x-zBvUPXkihzEk7mAJ5TQ
ImmutableMultiDict([('code', 'AQA7pz5PKsWkusLmU3KkE3Cc2KH7eMMzFkuil3Ns6jbBpQIXXlFqGKG9DLE66cWcTUObuPMDSGTb7fgL9vjxrDwaIuvuw9pcqQmLYNuwwOjQ_4ZCAvpggnxoyqMALSxV3oCcIHa-uqL4tv4ajnGkghGEM-Ns8F6s2zgXfbXQMMX87YEpcjObpzR8UIfJtJQ935NoLO7MsLpAvwj4VetT2k849MAagQ'), ('state', '3x-zBvUPXkihzEk7mAJ5TQ')])


127.0.0.1 - - [03/Nov/2023 12:00:17] "GET /callback?code=AQA7pz5PKsWkusLmU3KkE3Cc2KH7eMMzFkuil3Ns6jbBpQIXXlFqGKG9DLE66cWcTUObuPMDSGTb7fgL9vjxrDwaIuvuw9pcqQmLYNuwwOjQ_4ZCAvpggnxoyqMALSxV3oCcIHa-uqL4tv4ajnGkghGEM-Ns8F6s2zgXfbXQMMX87YEpcjObpzR8UIfJtJQ935NoLO7MsLpAvwj4VetT2k849MAagQ&state=3x-zBvUPXkihzEk7mAJ5TQ HTTP/1.1" 200 -


{'access_token': 'BQCM8U47rveoi5MOSpHblSOcCGsgYt9WOZhK-NGKnlJmK8UTdi8gc_oCS6V-mqlEfCHgA04hz-Fc3HxwRSZ3ONaYt_9jx1VRBeZBGUy2Sy1gJ5cNIkwMSVle95YHQNUGSxx3JvcbZxnFsVD9CuH7B0GwYOeReUvpUB7jLS4U_0yb1i5YjhtuOMhDbGvVlu0', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQAGjB_qLDk_-XBlrj6aygqVkm1IEm5k0tOFE-33ZiWn7s8t8hos4S-BfbwiAjKXyWI82VdUbi-LUrykG14BBDmS9Yvch-FbPX3Thy6vi89utGLsurVAEanhQSAjZLa3c_k', 'scope': 'user-read-email user-read-private'}


127.0.0.1 - - [03/Nov/2023 12:00:24] "GET /playlists HTTP/1.1" 200 -
